# Initialize
This section is a developer aid; TODO remove later!
*Align with [storyboard Version 37](https://rubinobs.atlassian.net/wiki/pages/viewpage.action?pageId=132612364&pageVersion=37)*

If you are using this under Time Square and *get any errors* that you think come from this notebook, please tell Steve Pothier (steve.pothier@noirlab.edu).  Include the URL you used to get the error. 

In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "YESTERDAY"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

In [2]:
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [3]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", ut.Server.usdf) # TODO try with "usdf" before push (else "summit")

In [4]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
limit = 5000  # YAGNI for Auto get more if this isn't enough to get all requested DAYS
response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

# Experimental Features config
exp_tally = "true"  # default: true
exp_parsing = "true"  # default: true
experimental = dict()
experimental["exposure_type_tally"] = exp_tally == "true"
experimental["error_parsing"] = exp_parsing == "true"

In [5]:
# Read records from (almost) all sources
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=False,                # TODO change to False before push
    limit=limit,
)
allrep = AllReports(allsrc=allsrc)

In [6]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")

# Showing data for 2024-11-03 to 2024-11-04 for LSSTComCamSim, LATISS, LSSTComCam, LSSTCam

# Table of Contents
* [Night Report](#Night-Report)
    - AuxTel
    - Simonyi
* [Almanac](#Almanac)
* [Summary plots of whole night](#Summary-plots-of-whole-night)
* [Time Accounting](#Time-Accounting)
* [Jira Tickets](#Jira-Tickets)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed)
    - AuxTel
    - Simonyi
* [Data Log](#Data-Log)
* [Narrative Log](#Narrative-Log)
* [Developer Only Section](#Developer-Only-Section)

## Night log (summary) <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

### Telescope: AuxTel

```
We had a bumpy start to the beginning of the observation day. During the bump tests, FA 235 failed several times, but we got confirmation that it was safe to raise the mirror. We had repeating issues with the MTHexapod today during its warmup. It failed several times, so we skipped the procedure - see OBS-620. We decided to reset the PXI for the M2 hexapod, and managed to initiate compensation mode. The TMA had some buggy behaviors as well. We tried to save some sky time and slew the TMA to the south well before sunset. Due to soft motion axis faults, and some weird but faultless behaviors (see details in the narrative log), it took around two hours to get the TMA into position. We went to dinner, came back, and prepared to open by sunset.

In addition to that afternoon fun, the ambient temperature went nuts starting around 2 hours before sunset, plummeting and rising again on a crazy roller coaster. 🎢 We opened just after sunset, fooled into thinking it had stabilized, and the M1M3 temperature dropped at an alarming rate of about 0.4 C per 15 minutes. We closed quickly, and turned on the HVAC systems to a 6 degree set point to try and keep M1M3 closer to ambient. Humidity began creeping up around 9:30 - reaching around 70%. We remained closed and continued in dome testing.

As the weather changed frequently, we opened the dome in steps to slowly thermalize the mirror while running the LUT. We decided to fully open around midnight and began on-sky testing. 

AOS night Summary:

Due to weather and temperature, we weren’t able to fully open the dome until ~00:30 local time. We used the time before this to perform T222 - Elevation LUT with closed dome.  Then we started T88 Laser Alignment to determine a good starting point. Using this starting point we started T92 to step though varying the M2 hexapod and camera hexapod dz positions to refine the focus.  First we moved the two hexapod shifts in the same direction and then had them move in different directions.  We found while doing this that we couldn’t return to the position we found in focus.  We determined this was due to the fact that we were not giving uncompensated positions.  We were using the compensated values and effectively applying them twice.  We built a new dashboard with uncompensated positions and then asked to go back to the uncompensated position via a run_command. It worked!  This explains why were weren’t able to reproduce values on previous nights.

With this information we once again tried the parameter march with the two hexapods moving in opposite directions.  We weren’t able to finish in time, and tried T202 which attempted to record the state of the system with the laser tracker.  We started the script and soon realized the system was not in focus. We stopped the script, but in fact the process did not end, and the laser tracker task completed and moved from a M1M3 SMR finally stopping at a Camera SMR.  When we next took an image to check focus, the extra light from the laser on the camera SMR tripped the voltage levels on the ComCom REBs, causing the ComCam to go into fault.  Brian Stalder remotely recovered the ComCam and we finished our tests due to the required closing time.
```

Confluence page: [testCycle/BLOCK-R126](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/testCycle/BLOCK-R126)

Telescope Status: 

*Authors: David Sanmartim, Alysha Shugart, Guillem Megias Homar, Chris Walter, Carlos Morales*

### Telescope: Simonyi

```
We had a bumpy start to the beginning of the observation day. During the bump tests, FA 235 failed several times, but we got confirmation that it was safe to raise the mirror. We had repeating issues with the MTHexapod today during its warmup. It failed several times, so we skipped the procedure - see OBS-620. We decided to reset the PXI for the M2 hexapod, and managed to initiate compensation mode. The TMA had some buggy behaviors as well. We tried to save some sky time and slew the TMA to the south well before sunset. Due to soft motion axis faults, and some weird but faultless behaviors (see details in the narrative log), it took around two hours to get the TMA into position. We went to dinner, came back, and prepared to open by sunset.

In addition to that afternoon fun, the ambient temperature went nuts starting around 2 hours before sunset, plummeting and rising again on a crazy roller coaster. 🎢 We opened just after sunset, fooled into thinking it had stabilized, and the M1M3 temperature dropped at an alarming rate of about 0.4 C per 15 minutes. We closed quickly, and turned on the HVAC systems to a 6 degree set point to try and keep M1M3 closer to ambient. Humidity began creeping up around 9:30 - reaching around 70%. We remained closed and continued in dome testing.

As the weather changed frequently, we opened the dome in steps to slowly thermalize the mirror while running the LUT. We decided to fully open around midnight and began on-sky testing. 

AOS night Summary:

Due to weather and temperature, we weren’t able to fully open the dome until ~00:30 local time. We used the time before this to perform T222 - Elevation LUT with closed dome.  Then we started T88 Laser Alignment to determine a good starting point. Using this starting point we started T92 to step though varying the M2 hexapod and camera hexapod dz positions to refine the focus.  First we moved the two hexapod shifts in the same direction and then had them move in different directions.  We found while doing this that we couldn’t return to the position we found in focus.  We determined this was due to the fact that we were not giving uncompensated positions.  We were using the compensated values and effectively applying them twice.  We built a new dashboard with uncompensated positions and then asked to go back to the uncompensated position via a run_command. It worked!  This explains why were weren’t able to reproduce values on previous nights.

With this information we once again tried the parameter march with the two hexapods moving in opposite directions.  We weren’t able to finish in time, and tried T202 which attempted to record the state of the system with the laser tracker.  We started the script and soon realized the system was not in focus. We stopped the script, but in fact the process did not end, and the laser tracker task completed and moved from a M1M3 SMR finally stopping at a Camera SMR.  When we next took an image to check focus, the extra light from the laser on the camera SMR tripped the voltage levels on the ComCom REBs, causing the ComCam to go into fault.  Brian Stalder remotely recovered the ComCam and we finished our tests due to the required closing time.

We have some issues to close the night, we can not send the MT Mount to enable again, the rotator was leave it in 23.2 position and off, we try the locking pins of TMA, but not insert complete so we move the TMA down en Elevation ~ 86 and the came back to 90 to try again, but we get an error when was parked at 90 degrees "Elevation Positive power off limit switch (TMA-EL-IS-CPD-0005) [sdiLIMELPOS]"
```

Telescope Status: MTM1M3 DISABLED - lowered
MTM2 ENABLED in closed loop
MTAOS STANDBY
MTDome DISABLED
MTDomeTrajectory STANDBY
MTMount DISABLED
MTHexapods in STANDBY w/ cabinets and PXIs powered off
MTPtg ENABLED
MTRotator at 23º, in STANDBY w/ cabinets and PXIs powered off
OSS ON
Power Supply ON
Locking Pins in TEST position
TMA position AZ= -32 EL=90.2

*Authors: David Sanmartim, Alysha Shugart, Guillem Megias Homar, Chris Walter, Carlos Morales*

## Almanac <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-11-03**

,(times in UTC),
Moon Rise,2024-11-03 10:51:23.690,
Moon Set,2024-11-03 00:22:54.999,
Moon Illumination,3%,(% illuminated)
Astronomical Twilight (morning),2024-11-03 08:18:35.948,(-18 degrees)
Astronomical Twilight (evening),2024-11-03 00:34:08.748,(-18 degrees)
Solar Midnight,2024-11-03 04:26:22.503,
Nautical Twilight (morning),2024-11-03 08:50:05.735,(-12 degrees)
Nautical Twilight (evening),2024-11-03 00:02:38.814,(-12 degrees)
Civil Twilight (morning),2024-11-03 09:20:25.007,(-6 degrees)
Civil Twilight (evening),2024-11-02 23:32:19.341,(-6 degrees)


## Summary plots of whole night <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

(content not yet defined in storyboard)

In [9]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

## Time Accounting <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

In [10]:
# Exposure Tally
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

## Jira Tickets <font style="background-color:green; color:white; font-size:20px">BETA</font> 
(Observing Operations - OBS)
(TODO: better tickets using Jira API)

In [11]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

- [/rubinobs-lfa-cp/LOVE/OLE/2024/11/03/LOVE_OLE_2024-11-03T21:41:04.106.png](https://s3.cp.lsst.org/rubinobs-lfa-cp/LOVE/OLE/2024/11/03/LOVE_OLE_2024-11-03T21:41:04.106.png)

- [OBS-627](https://rubinobs.atlassian.net/browse/OBS-627)

- [OBS-620](https://rubinobs.atlassian.net/browse/OBS-620)

- [OBS-641](https://rubinobs.atlassian.net/browse/OBS-641)

In [12]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

#### Nightly Jira BLOCKs
- 20241103
    - [testCycle/BLOCK-R126](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/testCycle/BLOCK-R126)

## BLOCKS Observed <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
(TODO: BLOCKS observed) 

## Data Log <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

NOTE: *modified all_sources.py:uniform_field_counts()*
NOTE: Each tally count is now a link to a detailed table of exposures.

In [13]:
# Exposure Report
if experimental.get("exposure_type_tally"): 
    #  md("### Experimental Exposure field Tally ENABLED")
    md("Tally exposures with specific values of selected fields")
else:
    md("### Experimental Exposure field Tally DISASBLED")
    
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    if 0 == len(allsrc.exp_src.exposures[instrum]):
        continue

    # Facet counts
    if experimental.get("exposure_type_tally"): 
        counts = allsrc.uniform_exposure_field_counts(instrum)
        for fname, tally in counts.items():
            md(f"{fname}")
            df = pd.DataFrame.from_dict(tally, orient="index").T
            wdf = ut.wrap_dataframe_columns(df)
            display(HTML(wdf.to_html(render_links=True, escape=False)))

Tally exposures with specific values of selected fields

**LSSTComCamSim: 0 total exposures**

**LATISS: 0 total exposures**

**LSSTComCam: 116 total exposures**

observation_reason

...

,flat,dark,sflat lo,preimage,sflat hi
0,2345,80,10,21,52


observation_type

,flat,bias,dark
0,1244,621,643


science_program

,E2016,E2012,E2011,E2014,E2015,E2013,E2010,E2009
0,2365,23,33,14,23,33,4,13


### (change) <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
#### TODO According to Storybaord, this section should be removed ...
... and replaced with a simple table tally of Good, Questionable, and  Junk exposures per instrument. (that tally has been added below)
| Symbol | Meaning |
|:---|:---|
| <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>| Good |
| <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font> | Questionable |
| <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font> | Junk |


In [14]:
# Time Log
allrep.exp_rep.time_log_as_markdown()

Note: *Added counts of Exposures with messages*

In [15]:
for instrum, recs in allsrc.exp_src.messages.items():
    md(f'**{instrum} {len(recs)} Exposures with messages**')
    if len(recs) > 0:
        df = allsrc.tally_exposure_flags(instrum)
        display(df.style.hide(axis="index"))

## Narrative Log <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [16]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

- **2024-11-03 08:58:43.457222** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Trying to get M2 back in closed loop state.</pre>


- **2024-11-03 08:50:56.147457** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Parked telescope, closed mirror covers. Accidentally powered off M2 cabinet, need to recover to put back in closed loop mode.</pre>


- **2024-11-03 07:47:47.831061** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Pinhole images taken, going to park for the night.</pre>


- **2024-11-03 07:45:19.428596** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>M1M3 faulted due to a excessive forces. This was user error in AOS calculations. M1M3 behaved propoerly</pre>


- **2024-11-03 07:44:34.305411** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

Failed to close mirror covers using run_command script


Error in run
<pre style='color: black; background-color: #FFDDDD; font-size: 0.875em; margin-left: 20px; '>Traceback (most recent call last):
  File &quot;/opt/lsst/software/stack/conda/envs/lsst-scipipe-9.0.0/lib/python3.11/site-packages/lsst/ts/salobj/base_script.py&quot;, line 631, in do_run
    await self._run_task
  File &quot;/net/obs-env/auto_base_packages/ts_standardscripts/python/lsst/ts/standardscripts/run_command.py&quot;, line 170, in run
    await getattr(self.remote, f&quot;cmd_{self.cmd}&quot;).start(
  File &quot;/opt/lsst/software/stack/conda/envs/lsst-scipipe-9.0.0/lib/python3.11/site-packages/lsst/ts/salobj/topics/remote_command.py&quot;, line 487, in start
    return await cmd_info.next_ackcmd(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File &quot;/opt/lsst/software/stack/conda/envs/lsst-scipipe-9.0.0/lib/python3.11/site-packages/lsst/ts/salobj/topics/remote_command.py&quot;, line 191, in next_ackcmd
    raise base.AckError(msg=&quot;Command failed&quot;, ackcmd=ackcmd)
lsst.ts.salobj.base.AckError: msg=&#x27;Command failed&#x27;, ackcmd=(ackcmd private_seqNum=252964407, ack=&amp;lt;SalRetCode.CMD_FAILED: -302&amp;gt;, error=1, result=&#x27;Failed: Command with id 2910388 has timed out after not receiving accept for 500 ms.&#x27;)


Ran the script again and it worked. 
</pre>


...

- **2024-11-03 18:45:15.505763** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Attempting hexapod warmups.</pre>


- **2024-11-03 18:26:43.483759** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>M2 warm up failed while exercising the y axis.</pre>


- **2024-11-03 17:45:20.284078** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>We are now starting to warm up hexapods.</pre>


- **2024-11-03 17:38:22.492453** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>FA 235 failed three times.</pre>


- **2024-11-03 17:38:08.841329** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Just after arriving at the control room, we started to power up hexapods and rotator. And we started bump tests.</pre>


-----------
------------

# Developer Only Section <font style="background-color:red; olor:black; font-size:20px"> REMOVE</font>
Contains stuff only expected to be useful to developers.

This may also contain sections that have moved out of the user section because they are no longer defined in the Storyboard.

## Overview 


In [17]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
""")

## Project-Wide Night(s) Report 
- Run on logs and databases from **https://usdf-rsp-dev.slac.stanford.edu/**
- Report **1 observing night(s)** with the last reported night starting on **2024-11-03 12:00:00**.
- min_dayobs='2024-11-03', max_dayobs='2024-11-04'
- Using ***Prototype* Logging and Reporting** Version: **0.1.dev145+gf2a5765.d20241014**
- have_consdb = True
- Imported lsst.ts.logging_and_reporting from local packages.


## Data Status
This may be useful for diagnosing reports that give unexpected results.

In [18]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(
    index=False, 
    render_links=True,
    escape=False)
display(HTML(table))

Endpoint,Records,URL
nightreport/reports,2,https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241103&max_day_obs=20241104
exposurelog/instruments,3,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/instruments
exposurelog/exposures.LSSTComCamSim,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241103&max_day_obs=20241104
exposurelog/exposures.LATISS,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LATISS&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241103&max_day_obs=20241104
exposurelog/exposures.LSSTComCam,116,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241103&max_day_obs=20241104
exposurelog/exposures.LSSTCam,2508,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTCam&order_by=-timespan_end&offset=2500&limit=2500&min_day_obs=20241103&max_day_obs=20241104
exposurelog/messages,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=2500&min_day_obs=20241103&max_day_obs=20241104
narrativelog/messages,74,https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-03T00%3A00%3A00&max_date_added=2024-11-04T00%3A00%3A00
efd/targets,0,NA


## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## DDV 

In [19]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

Access DDV part of RubinTV: https://usdf-rsp-dev.slac.stanford.edu/rubintv/ddv/index.html

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [20]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

### Overview for Service: `nightreport` [2]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241103&max_day_obs=20241104

None
- Got 2 records.  


### Overview for Service: `exposurelog` [0]

...

None
- Got 0 records.  


### Overview for Service: `narrativelog` [74]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-03T00%3A00%3A00&max_date_added=2024-11-04T00%3A00%3A00

None
- Got 74 records.  


In [21]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

## Finale

In [22]:
print(f"Finished {str(dt.datetime.now())}")

Finished 2024-11-04 14:54:35.032233
